In [118]:
with open('input.txt') as f:
    input = f.read().splitlines()

In [119]:
import re

def transformInput(input):

    rules = []
    for row in input:
        ruleRegex = r"^(.*) bags contain (.*) bags?\."
        ruleMatches = re.findall(ruleRegex, row)[0]

        subject = ruleMatches[0]
        allowedContents = ruleMatches[1]
        
        # If no bags allowed ignore the rule
        if allowedContents == "no other":
            continue

        allowedBagsData = allowedContents.split(', ')
        allowedBags = []
    
        for allowedBagData in allowedBagsData:
            bagWithAmount = allowedBagData.replace(' bags', '').replace(' bag', '')

            allowedBagRegex = r"([0-9]+) (.*)"
            allowedBagMatches = re.findall(allowedBagRegex, bagWithAmount)[0]

            allowedBags.append({ 'amount': int(allowedBagMatches[0]), 'bag': allowedBagMatches[1] })
            
        rules.append({ 
            'subject': subject, 
            'allowedBags': allowedBags
        })

    return rules

def flattenRules(rules):
    flatRules = []

    for rule in rules:
        for bag in rule['allowedBags']:
            flatRules.append({ 
                'subject': rule['subject'],
                'allowedBag': bag['bag']
            })

    return flatRules

def findRuleForBag(rules, bag):
    for rule in rules:
        if rule['subject'] == bag:
            return rule

    return None


# This could/should be way more optimized! There is no white/black list of (in)valid items.
def canContainBag(rules, subject, path, allowed, search):
    # Return true if allowed child matches with search
    if allowed == search:
        return True

    path.append(subject)

    # Don't check childs if it's already checked
    if allowed in path:
        return False

    for rule in rules:
        # Don't process other bags that aren't allowed
        if rule['subject'] != allowed:
            continue
        
        # Lets check it
        if canContainBag(rules, rule['subject'], path, rule['allowedBag'], search):
            return True
    
    return False

def countContainingBags(rules, bag):
    result = 1
    
    rule = findRuleForBag(rules, bag)

    if rule is None:
        return result

    for innerBag in rule['allowedBags']:
        result += innerBag['amount'] * countContainingBags(rules, innerBag['bag'])

    return result

## Get results

In [120]:
result1 = 0
result2 = 0

validBags = []

# Parse input rules
rules = transformInput(input)

# Flatten with multiple inner bags to single bags for part 1
flatRules = flattenRules(rules)

# Part 1
for rule in flatRules:
    # Skip checking valid bags
    if rule['subject'] in validBags:
        continue

    if canContainBag(flatRules, rule['subject'], [ ], rule['allowedBag'], 'shiny gold'):
        validBags.append(rule['subject'])
        result1 += 1
    

# Part 2
result2 = countContainingBags(rules, 'shiny gold') - 1

print(f"Part 1: {result1}")
print(f"Part 2: {result2}")

Part 1: 252
Part 2: 35487
